## CatBoost Hyperparameter Tuning (Recall)

In [10]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from catboost import CatBoostClassifier

import optuna

In [11]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [14]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [20]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'iterations': trial.suggest_int('iterations', 500, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 6, 16),
        'random_strength': trial.suggest_float('random_strength', 0.3, 1),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2.0, 10.0)
    }

    model = CatBoostClassifier(
        **params,
        verbose=0,
        random_state=42)
    
    threshold = 0.29
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [22]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 23:23:12,643] A new study created in memory with name: no-name-275468a2-d8a2-4356-9483-890c29739ae8


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 23:25:06,432] Trial 0 finished with value: 0.5539877300613496 and parameters: {'iterations': 896, 'learning_rate': 0.04717034219365293, 'depth': 14, 'random_strength': 0.8797714361731837, 'bagging_temperature': 0.9920666119693674, 'l2_leaf_reg': 4.191741545235302}. Best is trial 0 with value: 0.5539877300613496.
[I 2024-07-08 23:25:38,676] Trial 1 finished with value: 0.5687116564417178 and parameters: {'iterations': 502, 'learning_rate': 0.06823524196549374, 'depth': 13, 'random_strength': 0.5623614949528171, 'bagging_temperature': 0.491102258505801, 'l2_leaf_reg': 7.916810708323666}. Best is trial 1 with value: 0.5687116564417178.
[I 2024-07-08 23:25:50,232] Trial 2 finished with value: 0.5957055214723926 and parameters: {'iterations': 939, 'learning_rate': 0.028985324053387343, 'depth': 9, 'random_strength': 0.906565002666909, 'bagging_temperature': 0.5326567053418519, 'l2_leaf_reg': 7.293269291753438}. Best is trial 2 with value: 0.5957055214723926.
[I 2024-07-08 23:3

In [28]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=41, state=1, values=[0.6496932515337422], datetime_start=datetime.datetime(2024, 7, 9, 0, 14, 37, 241149), datetime_complete=datetime.datetime(2024, 7, 9, 0, 14, 42, 525641), params={'iterations': 598, 'learning_rate': 0.023443878098496275, 'depth': 7, 'random_strength': 0.5850280189408111, 'bagging_temperature': 0.8338604000665841, 'l2_leaf_reg': 8.631900728051564}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'iterations': IntDistribution(high=1000, log=False, low=500, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'depth': IntDistribution(high=16, log=False, low=6, step=1), 'random_strength': FloatDistribution(high=1.0, log=False, low=0.3, step=None), 'bagging_temperature': FloatDistribution(high=1.0, log=False, low=0.1, step=None), 'l2_leaf_reg': FloatDistribution(high=10.0, log=False, low=2.0, step=None)}, trial_id=41, value=None)
Best hyperparameters: {'iterations': 598, 'learning_rat